In [1]:
from ilqr import iLQR
import gym
import numpy as np
import pandas as pd
import time

from aprl.agents import MujocoFiniteDiffDynamicsPerformance
from aprl.envs.mujoco_costs import get_cost
from experiments.common import set_seeds, make_env, fit_ilqr, \
                               on_iteration, receding, multi_evaluate

WARNING (theano.configdefaults): install mkl with `conda install mkl-service`: No module named 'mkl'


Logging to /tmp/openai-2019-02-03-10-03-49-574699
Choosing the latest nvidia driver: /usr/lib/nvidia-396, among ['/usr/lib/nvidia-375', '/usr/lib/nvidia-396']
Choosing the latest nvidia driver: /usr/lib/nvidia-396, among ['/usr/lib/nvidia-375', '/usr/lib/nvidia-396']


In [25]:
from ilqr.cost import BatchAutoDiffCost
from theano import tensor as T

class DemoCost(BatchAutoDiffCost):
    def __init__(self, axis):
        def f(x, u, i, terminal):
            if terminal:
                control_cost = T.zeros_like(x[..., 0])
            else:
                control_cost = T.square(u).sum(axis=axis)
            
            foo = x[..., 0:1]
            state_cost = T.sqrt(T.sum(T.square(foo), axis=axis))
            cost = state_cost + control_cost
            return cost

        super().__init__(f, state_size=2, action_size=2)

class WorkingCost(BatchAutoDiffCost):
    """Differentiable cost for the Reacher-v2 Gym environment.
       See base class for more details."""
    def __init__(self):
        def f(x, u, i, terminal):
            if terminal:
                ctrl_cost = T.zeros_like(x[..., 0])
            else:
                ctrl_cost = T.square(u).sum(axis=-1)

            # x: (batch_size, 8)
            # x[..., 0:4]: qpos
            # x[..., 4:8]: qvel, time derivatives of qpos, not used in the cost.
            theta = x[..., 0]  # qpos[0]: angle of joint 0
            phi = x[..., 1]  # qpos[1]: angle of joint 1
            target_xpos = x[..., 2:4]  # qpos[2:4], target x & y coordinate
            body1_xpos = 0.1 * T.stack([T.cos(theta), T.sin(theta)], axis=1)
            tip_xpos_incr = 0.11 * T.stack([T.cos(phi), T.sin(phi)], axis=1)
            tip_xpos = body1_xpos + tip_xpos_incr
            delta = tip_xpos - target_xpos

            state_cost = T.sqrt(T.sum(delta * delta, axis=-1))
            cost = state_cost + ctrl_cost

            return cost

        super().__init__(f, state_size=8, action_size=2)
        
class BrokenCost(BatchAutoDiffCost):
    """Differentiable cost for the Reacher-v2 Gym environment.
       See base class for more details."""
    def __init__(self):
        def f(x, u, i, terminal):
            if terminal:
                ctrl_cost = T.zeros_like(x[..., 0])
            else:
                ctrl_cost = T.square(u).sum(axis=-1)

            # x: (batch_size, 8)
            # x[..., 0:4]: qpos
            # x[..., 4:8]: qvel, time derivatives of qpos, not used in the cost.
            theta = x[..., 0]  # qpos[0]: angle of joint 0
            phi = x[..., 1]  # qpos[1]: angle of joint 1
            target_xpos = x[..., 2:4]  # qpos[2:4], target x & y coordinate
            body1_xpos = 0.1 * T.stack([T.cos(theta), T.sin(theta)], axis=1)
            tip_xpos_incr = 0.11 * T.stack([T.cos(phi), T.sin(phi)], axis=1)
            tip_xpos = body1_xpos + tip_xpos_incr
            delta = tip_xpos - target_xpos

            state_cost = T.sqrt(T.sum(delta * delta, axis=0))
            cost = state_cost + ctrl_cost

            return cost

        super().__init__(f, state_size=8, action_size=2)

In [26]:
wcost = DemoCost(axis=-1)
bcost0 = DemoCost(axis=0)
try:
    bcost1 = DemoCost(axis=1)
except Exception as e:
    print('DemoCost(axis=1) failed with', e)

ERROR (theano.gof.opt): Optimization failure due to: constant_folding
ERROR (theano.gof.opt): node: Elemwise{mul,no_inplace}(TensorConstant{[[1.]
 [0.]]}, TensorConstant{(1, 2) of 1.0})
ERROR (theano.gof.opt): TRACEBACK:
ERROR (theano.gof.opt): Traceback (most recent call last):
  File "/home/adam/bin/anaconda3/envs/aprl/lib/python3.6/site-packages/theano/gof/opt.py", line 2034, in process_node
    replacements = lopt.transform(node)
  File "/home/adam/bin/anaconda3/envs/aprl/lib/python3.6/site-packages/theano/tensor/opt.py", line 6518, in constant_folding
    required = thunk()
  File "/home/adam/bin/anaconda3/envs/aprl/lib/python3.6/site-packages/theano/gof/op.py", line 862, in rval
    thunk()
  File "/home/adam/bin/anaconda3/envs/aprl/lib/python3.6/site-packages/theano/gof/cc.py", line 1735, in __call__
    reraise(exc_type, exc_value, exc_trace)
  File "/home/adam/bin/anaconda3/envs/aprl/lib/python3.6/site-packages/six.py", line 693, in reraise
    raise value
ValueError: Input di

DemoCost(axis=1) failed with Not enough dimensions on Elemwise{sqr,no_inplace}.0 to reduce on axis 1


In [22]:
import numpy as np

x = np.random.rand(8)
u = np.random.rand(2)

In [27]:
def similar(x, y):
    x = abs(x - y) < 1e-8
    if np.isscalar(x):
        return x
    else:
        return x.all()
print('l', similar(wcost.l(x, u, [0]), bcost0.l(x, u, [0])))
print('l_x', similar(wcost.l_x(x, u, 0), bcost0.l_x(x, u, 0)))
print('l_u', similar(wcost.l_u(x, u, 0), bcost0.l_u(x, u, 0)))
print('l_xx', similar(wcost.l_xx(x, u, 0), bcost0.l_xx(x, u, 0)))
print('l_uu', similar(wcost.l_uu(x, u, 0), bcost0.l_uu(x, u, 0)))

l True
l_x True
l_u True
l_xx False
l_uu True


In [24]:
wcostf = WorkingCost()
bcostf = BrokenCost()
print('l', similar(wcostf.l(x, u, [0]), bcostf.l(x, u, [0])))
print('l_x', similar(wcostf.l_x(x, u, 0), bcostf.l_x(x, u, 0)))
print('l_u', similar(wcostf.l_u(x, u, 0), bcostf.l_u(x, u, 0)))
print('l_xx', similar(wcostf.l_xx(x, u, 0), bcostf.l_xx(x, u, 0)))
print('l_uu', similar(wcostf.l_uu(x, u, 0), bcostf.l_uu(x, u, 0)))

l True
l_x True
l_u True
l_xx False
l_uu True


In [8]:
xs, us = fit_ilqr(ilqrs, x0s, us_init)

dict_keys(['performance'])
*** Fitting performance ***
iteration 0 accepted 8.861193226544234 [-3.52186781  0.69727993  0.02243766  0.07369059 -8.35248998 -0.61676626
  0.          0.        ] [-0.05328001 -0.14284116]
iteration 1 accepted 8.266717338733763 [-3.49583825  0.7658604   0.02243766  0.07369059 -8.32863992 -0.49937366
  0.          0.        ] [-0.01209801 -0.05909925]
iteration 2 accepted 8.104386993643296 [-3.32992403  0.87136392  0.02243766  0.07369059 -8.09216039 -0.36647474
  0.          0.        ] [-0.0295018  -0.01665231]
iteration 3 accepted 7.518675510352434 [-1.72845306  1.83136225  0.02243766  0.07369059 -5.28381334  1.47827829
  0.          0.        ] [-0.04382504 -0.02880142]
iteration 4 failed 7.518675510352435 [-1.72845306  1.83136225  0.02243766  0.07369059 -5.28381334  1.47827829
  0.          0.        ] [-0.04382504 -0.02880142]
iteration 5 accepted 7.514264455609661 [-1.6299537   1.87320724  0.02243766  0.07369059 -4.93110861  1.61710136
  0.          0

# Receding horizon

In [ ]:
for k, ilqr in ilqrs.items():
    underlying = 'receding_' + k
    dynamics[underlying] = dynamics[k]
    x0s[underlying] = x0s[k]
    xs[underlying], us[underlying] = receding(ilqr, x0s[k], us_init, seed=seed)

# Rollouts

In [ ]:
multi_evaluate(env, dynamics, x0s, us)